# Esercizio 2 - Content2Word

In questo esercizio cerchiamo di risalire al termine partendo dalle definizioni date a lezioni e che abbiamo già utilizzato nell'esercizio 1.
Visionare "es1_defs" per ulteriori approfondimenti sul dataset.

### Approccio

- Prendiamo i terminini più frequenti nelle definizioni, che saranno i genus.
  *Perchè i Genus?* Così possiamo restringere la ricerca in wordnet, confrontando le nostre definizioni con un sottoinsieme di definizioni
  al posto di tutte le definizioni di wordnet, passando da migliaia di definizioni ad un centinaio, in modo da rendere la ricerca più efficiente.
  
- *Text cleaning*: Stopwords removing e lemmatization
- Preleviamo tutto il sottoalbero di iponimi dei genus
- Prendiamo le definizioni (glossa) dei synset, ricavati dagli iponimi trovati al passo precedente
- Facciamo il confronto tra definizioni di wordnet e la nostra lista di definizioni
  *Come confontiamo le definizioni?* Andiamo a scegliere come termine quello che nella definizione ha più parole in comune con le nostre definizioni.
- Restituisco il synset che ha òa definizione più simile a quella della lista

### Imports

In [1]:
from nltk.corpus import stopwords
from collections import Counter
from gensim.test.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
import random

### Methods

In [2]:
def get_text_from_file(path):
    '''
    Read a file and, after revoving all stopwords, return a list of words.
    '''
    file = []
    stop_words = set(stopwords.words('english'))
    with open (path, 'r') as f:
        for row in f:
            filtered_s = [w for w in word_tokenize(row) if not w.lower() in stop_words]
            file.append(simple_preprocess(str(filtered_s), deacc=True))
    f.close()
    return file

def get_most_freq_words(text, nword):
    '''
    Given a list of sententeces, return the nword most frequent words
    in each row of the document.
    '''
    genus = []
    for row in text:
        c = Counter()
        c.update(row)
        genus.append(c.most_common(nword))
    return genus

def get_hypos(word):
    '''
    Return all the hyponyms of a word.
    '''
    syn = get_synset(word)
    hypo_list = []
    if(syn is not None):
        hypo_list = list(set([w for s in syn.closure(lambda s:s.hyponyms()) for w in s.lemma_names()]))
    return hypo_list

def get_hypers(word):
    '''
    Return the direct hypernyms of a word.
    '''
    syn = get_synset(word)
    hypo_list = []
    if(syn is not None):
        hypo_list = list(set([w for s in syn.closure(lambda s:s.hypermyms()) for w in s.lemma_names()]))
    return hypo_list

def get_synset(word):
    '''
    Retrurn the first synset of a word.
    '''
    if(len(wn.synsets(word)) > 0):
        return wn.synsets(word)[0]
    return None


### Define constants

Andiamo a modificare questi due valori si ottengono risultati diversi. 
Dopo alcuni test abbiamo scelto questi valori siccome erano quelli che retituivano risultati migliori.

In [3]:
num_genus = 3 # number of most frequent words to search in the definitions file, used to determine genus
num_most_freq_word = 10 # number of most frequent words to search in the definitions, used to compare with the wordnet's definitions

starting_words = ["Emotion", "Person", "Revenge", "Brick"]

### Pre-processing data and find the genus

In [4]:
file = get_text_from_file('../data/def.csv')

genus = get_most_freq_words(file, num_genus)

print (genus)

[[('feeling', 11), ('human', 8), ('feel', 8)], [('human', 26), ('person', 5), ('homo', 5)], [('someone', 14), ('feeling', 7), ('anger', 7)], [('used', 22), ('object', 15), ('material', 13)]]


Utilizziamo 3 genus in modo da aumentare l'accuratezza:

In [5]:
genus_list = []
for el in genus:
    genus_list_inner = []
    for el2 in el:
        genus_list_inner.append(el2[0])
    genus_list.append(genus_list_inner)
        
genus_list

[['feeling', 'human', 'feel'],
 ['human', 'person', 'homo'],
 ['someone', 'feeling', 'anger'],
 ['used', 'object', 'material']]

### Main

In [7]:
# Extract the most used word in the definitions
key_words_defs = get_most_freq_words(file, num_most_freq_word)

for i in range(len(genus_list)):
    
    # Top 10 word used in the definitions
    key_row = []
    for el in key_words_defs[i]:
        key_row.append(el[0])
    
    #* Version with 1 genun
    # Get the hyponyms of the genus and find the definition of the hyponyms
    # hypo_list = get_hypos(genus_list[i])
    # print(hypo_list)
    # hypo_def = []
    # for hypo in hypo_list:
    #     hypo_def.append((hypo, get_synset(hypo).definition()))
    
    #* version with multiple genus
    hypo_list, hypo_def = [], []
    for el in genus_list[i]:
        hypo_list.append(get_hypos(el))
        
    hypo_list = [x for xs in hypo_list for x in xs]
    for hypo in hypo_list:
        hypo_def.append((hypo, get_synset(hypo).definition()))
    
    # Compare the definition of our definitions (def.csv file) with the definition of the hyponyms
    res = []
    for wndef in hypo_def: # Definition of the hyponyms in wordnet
        score = 0
        imp_words = []
        for key_word in key_row: # Definition given by us
            if(key_word in wndef[1]):
                score += 1
                imp_words.append(key_word)      
        
        # Store all the value
        res.append((score, wndef[0], imp_words, wndef[1]))
        
    sorted_list = sorted(res, key=lambda x: x[0])
    sorted_res = list(reversed(sorted_list))
    print("\nOriginal Word:", starting_words[i], "/  Genus -->",genus_list[i])
    for k in range(min(len(sorted_res), 5)):
        #* Long print
        # print(f'Word: *{sorted_res[k][1]}*, score: *{sorted_res[k][0]}*, the key words are *{sorted_res[k][2]}* and the definition is *{sorted_res[k][3]}*')
        print(f'Word: *{sorted_res[k][1]}*, score: *{sorted_res[k][0]}*')


Original Word: Emotion /  Genus --> ['feeling', 'human', 'feel']
Word: *appetite*, score: *3*
Word: *boredom*, score: *3*
Word: *wonderment*, score: *3*
Word: *disapproval*, score: *3*
Word: *relief*, score: *3*

Original Word: Person /  Genus --> ['human', 'person', 'homo']
Word: *Paleo-American*, score: *2*
Word: *Khanty*, score: *2*
Word: *Cochimi*, score: *2*
Word: *Kui*, score: *2*
Word: *Basque*, score: *2*

Original Word: Revenge /  Genus --> ['someone', 'feeling', 'anger']
Word: *sounding_board*, score: *5*
Word: *stolidity*, score: *4*
Word: *hate*, score: *4*
Word: *unemotionality*, score: *4*
Word: *impassivity*, score: *4*

Original Word: Brick /  Genus --> ['used', 'object', 'material']
Word: *brick*, score: *5*
Word: *building_material*, score: *4*
Word: *kaolin*, score: *3*
Word: *writing_paper*, score: *3*
Word: *manure*, score: *3*


## Analisi dei risultati

Nella stampa sopra abbiamo i genus trovati e i relativi iponimi con grado di similarità maggiore.

Come possiamo vedere l'algoritmo non lavora bene, infatti riesce solo a trovare *brick* e lo trova in prima posizione (bene!). 
Per i restanti non ci va nemmeno vicino, infatti non compaiono i termini di riferimento nei primi 5 risultati, cerchiamo di capire come mai:

1. **I genus**: Come possiamo vedere i genus di riferimento hanno poco a che vedere con i termini, come ad esempio *"someone"* per *revenge*.
   Questo è portato dal dataset in input. Si potrebbe ripulire il dataset rimuovendo le parole che non hanno a che vedere con il termine originale,
   ma così facendo si andrebbe a compromettere l'esercizio e si renderebbe questo approccio poco scalabile su altre basi di dati.

2. **Iponimi**: Non è detto che il termine che stiamo cercando sia un suo iponimo dei genus che abbiamo selezioanto, infatti potrebbe essere 
   un iperonimo o essere proprio in un punto completamente diverso dell'albero di wordnet. Un possibile miglioramento dell'algoritmo potrebbe
   essere quello di andare a prelevare altri synset oltre agli iponimi del genus, ad esempio andando a prelevare anche i fratelli del genus,
   senza allontarsi troppo per non far esplodere la complessità dell'algoritmo.

3. **Funzione di similarità**: Potremmo decidere anche di utilizzare altre funzioni di similarità oltre a quella basata sulla comparazione dei termini 
   più frequenti. L'algoritmo è basato su uno *score* che corrisponde a quante parole simili ci sono nelle definizioni. Potremmo andare ad 
   aumentare lo *score* sulla base di altri fattori, come la funzione di similarità di wordnet *path_similarity*.

Infine, anche se i risultati non sono ottimi, non ci stupiamo del fatto che siamo riusciti a trovare *brick* siccome, almeno in teoria, è il
termine più facile a cui dare una definizione.